In [260]:
import pandas as pd
import json
from pandas import json_normalize

In [261]:
# Load JSON
f = open('./SpotifyPlaylists/mpd.slice.0-999.json')  
data = json.load(f)
f.close()

In [262]:
# Convert playlists array to df
df = pd.DataFrame(data['playlists'])
df.head()

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16


In [263]:
# Clean unnecessary columns
df_cleaned = df.filter(['name','tracks'], axis=1)
df_cleaned.head()

,name,tracks
0,Throwbacks,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't..."
1,Awesome Playlist,"[{'pos': 0, 'artist_name': 'Survivor', 'track_..."
2,korean,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri..."
3,mat,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën..."
4,90s,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk..."


In [264]:
# Create final df with nested track's array in each playlist
df_final = pd.DataFrame()
for i in range(0, len(data['playlists'])):
    df_nested_list = pd.json_normalize(
        data['playlists'][i], 
        record_path =['tracks'],
        meta=['name']
    )
    df_nested_list.drop_duplicates(subset=['track_name'])
    df_cleaned2 = df_nested_list.filter(['name','album_name','track_name','artist_name'], axis=1)
    df_final = pd.concat([df_final, df_cleaned2], ignore_index=True)
df_final.rename(columns = {'name':'list_name'}, inplace = True)
df_final = df_final.filter(['list_name','track_name'], axis=1)
df_final['score'] = 1
df_final.head(50)

,list_name,track_name,score
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop),1
1,Throwbacks,Toxic,1
2,Throwbacks,Crazy In Love,1
3,Throwbacks,Rock Your Body,1
4,Throwbacks,It Wasn't Me,1
5,Throwbacks,Yeah!,1
6,Throwbacks,My Boo,1
7,Throwbacks,Buttons,1
8,Throwbacks,Say My Name,1
9,Throwbacks,Hey Ya! - Radio Mix / Club Mix,1


In [265]:
#Remove duplicated values en each list
df_final.drop_duplicates(subset=['list_name', 'track_name'], keep='first', inplace=True)
ids = df_final["track_name"]
df_final[ids.isin(ids[ids.duplicated()])].sort_values("track_name").head(50)

,list_name,track_name,score
39,Throwbacks,All The Small Things,1
756,90's,All The Small Things,1
850,New Songs,Amsterdam,1
981,Mom's playlist,Amsterdam,1
1237,fall '17,Another Day in Paradise,1
1289,run it,Another Day in Paradise,1
446,2017,Attention,1
1234,fall '17,Attention,1
1365,smooth,Believe,1
1001,Mom's playlist,Believe,1


In [266]:
# Convert df in oneHotEncoded for tracks
from sklearn.preprocessing import LabelBinarizer

jobs_encoder = LabelBinarizer()
jobs_encoder.fit(df_final['track_name'])
transformed = jobs_encoder.transform(df_final['track_name'])
ohe_df = pd.DataFrame(transformed)
data_ohe = pd.concat([df_final, ohe_df], axis=1).drop(['track_name'], axis=1)
data_ohe.head()

,list_name,score,0,1,2,3,4,5,6,7,...,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382
0,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [267]:
# Convert df in oneHotEncoded for tracks with track_names as columns (Cleaning name)
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(sparse=False)
train_X_encoded = pd.DataFrame(encoder.fit_transform(df_final[['track_name']]))
train_X_encoded.columns = encoder.get_feature_names_out(['track_name'])
for index in range(0, len(train_X_encoded.columns)):
    string = str(train_X_encoded.columns[index])
    replace = string.replace('track_name_','')
    train_X_encoded.columns.values[index] = replace
df_final.drop(['track_name'] ,axis=1, inplace=True)
OH_X_train= pd.concat([df_final, train_X_encoded ], axis=1)
OH_X_train.head()

,list_name,score,$ave Dat Money (feat. Fetty Wap & Rich Homie Quan),(Everything I Do) I Do It For You,(Not So) Sad And Lonely,1 Billion Dogs,1-800-273-8255,10:1,13,19,...,pick up the phone,suncream,way it goes,wokeuplikethis*,wonderful,꺼내 먹어요 (Eat),두근거려 (Beautiful),사실은 The Truth Is,양화대교 (Yanghwa Brdg),전야 前夜 The Eve
0,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Throwbacks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [268]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_ohe, test_size=0.2)
train.shape

(1157, 1385)

In [269]:
test.shape

(290, 1385)

In [270]:
# example of one hot encoding for a neural network
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense

# load the dataset
def load_dataset(filename):
	# load the dataset as a pandas DataFrame
	data = read_csv(filename, header=None)
	# retrieve numpy array
	dataset = data.values
	# split into input (X) and output (y) variables
	X = dataset[:, :-1]
	y = dataset[:,-1]
	# format all fields as string
	X = X.astype(str)
	# reshape target to be a 2d array
	y = y.reshape((len(y), 1))
	return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
	ohe = OneHotEncoder()
	ohe.fit(X_train)
	X_train_enc = ohe.transform(X_train)
	X_test_enc = ohe.transform(X_test)
	return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

# load the dataset
#X, y = load_dataset('breast-cancer.csv')
# split into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
train, test = train_test_split(OH_X_train, test_size=0.2)
# prepare input data
#X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
#y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# define the  model
model = Sequential()
model.add(Dense(10, input_dim=train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(train, train, batch_size=64,epochs=2, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(test, test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).